In [4]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv

# --- 1. Load API Key ---
load_dotenv()
KOSIS_API_KEY = os.getenv("KOSIS_API_KEY")

# --- 2. Function to Get Data ---
def get_kosis_loan_rate(api_key):
    """Fetches loan rate data from KOSIS API and returns a DataFrame."""

    if not api_key:
        print("🚨 Error: KOSIS_API_KEY is not set in your .env file.")
        return pd.DataFrame() # Return an empty DataFrame

    # KOSIS API endpoint
    base_url = "https://kosis.kr/openapi/Param/statisticsParameterData.do"

    # --- ❗ Update these IDs from KOSIS Website ---
    table_id = "DT_YOUR_TABLE_ID"  # Replace with actual Table ID
    item_id = "YOUR_ITEM_ID"       # Replace with actual Item ID (e.g., average rate)
    # --- ❗ ---

    # API parameters
    params = {
        "method": "getList",
        "apiKey": api_key,
        "format": "json",
        "jsonVD": "Y",
        # "tblId": table_id,      # Use the variable
        # "objL1": item_id,       # Use the variable
        "prdSe": "M",           # Monthly data
        "startPrdDe": "201501", # Start date (YYYYMM)
        "endPrdDe": "202512",   # End date (YYYYMM)
    }
    
    # Check if placeholder IDs are still there
    if "YOUR_TABLE_ID" in table_id or "YOUR_ITEM_ID" in item_id:
        print("⚠️ Warning: Please update 'table_id' and 'item_id' with actual values from KOSIS.")
        # Optional: Stop execution until IDs are updated
        # return pd.DataFrame() 

    try:
        print("Requesting data from KOSIS API...")
        response = requests.get(base_url, params=params)
        response.raise_for_status() # Check for HTTP errors (like 404, 500)
        
        data = response.json()

        if not data or isinstance(data, dict): # Basic check if data seems invalid
             print("🚨 Error: No data received or API returned an error message.")
             # You might want to print 'data' here to see the error message from KOSIS
             # print("API Response:", data) 
             return pd.DataFrame()

        print("Data received, converting to DataFrame...")
        df = pd.DataFrame(data)

        # Process the data
        df['date'] = pd.to_datetime(df['PRD_DE'], format='%Y%m') # Convert YYYYMM to Date
        df['Loan_Rate'] = pd.to_numeric(df['DT'], errors='coerce') # Convert value 'DT' to number

        # Select and sort the final data
        final_df = df.set_index('date')[['Loan_Rate']].sort_index()
        
        print("✅ Success!")
        return final_df

    except requests.exceptions.RequestException as e:
        print(f"🚨 Network Error during API request: {e}")
    except Exception as e:
        print(f"🚨 An error occurred: {e}")
        
    return pd.DataFrame() # Return empty DataFrame on any error

# --- 3. Main Execution ---
if __name__ == "__main__":
    
    # Call the function to get the data
    loan_rate_data = get_kosis_loan_rate(KOSIS_API_KEY)
    
    # Print the latest 12 months if data was loaded successfully
    if not loan_rate_data.empty:
        print("\n--- Loan Rate Data (Latest 12 Months) ---")
        print(loan_rate_data.tail(12))
    else:
        print("\nFailed to load data.")

⚠️ Warning: Please update 'table_id' and 'item_id' with actual values from KOSIS.
Requesting data from KOSIS API...
🚨 Error: No data received or API returned an error message.

Failed to load data.
